In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD
import math
import pickle

In [ ]:
with open("../data/dowjones_calculated/periods.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [ ]:
X_train = dataset[0][0][0]
meanX = X_train.mean(axis=1)
stdX = X_train.std(axis = 1)
X_train = X_train.sub(meanX, axis=0)
X_train = X_train.div(stdX, axis = 0)
X_train.values

array([[-0.16818366, -0.06806208, -0.79488366, ...,  1.90748156,
        -0.13237066, -0.15001104],
       [ 1.18430514, -0.98796489,  0.64962518, ...,  0.96275308,
        -1.61805741, -0.72916989],
       [ 2.51184689,  1.27560147, -0.43122601, ..., -1.53742199,
        -0.46888445,  0.81014862],
       ...,
       [-1.79100716, -1.55909641, -0.87240762, ...,  1.09324552,
         0.973972  ,  0.68361082],
       [ 0.92562432,  1.04820182,  2.32825578, ..., -0.86449875,
        -0.40097858,  0.69698454],
       [-1.17604239, -1.89023378,  0.29202799, ..., -0.21530064,
        -0.12943477,  0.48714708]])

In [ ]:
X_train = dataset[0][0][0]
meanX = X_train.mean(axis=1)
stdX = X_train.std(axis = 1)
X_train = X_train.sub(meanX, axis=0)
X_train = X_train.div(stdX, axis = 0)
X_train = X_train.values
# X_train = dataset[0][0][0].values
# X_train = (X_train - X_train.mean())/X_train.std()
y_train = dataset[0][0][1].values

# returns = dataset[1][0][0]
# returns = (returns - returns.mean()) / returns.std()
X_test = dataset[1][0][0].values
y_test = dataset[1][0][1].values

In [ ]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])
y_train = get_one_hot(y_train, 2)
y_test = get_one_hot(y_test, 2)

In [ ]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")
# print(f"predicted shape: {predicted.shape}")

x train shape: (750, 31)
y train shape: (750, 31, 2)
x test shape: (490, 31)
y test shape: (490, 31, 2)


In [ ]:
data = X_train
targets = y_train

train_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=510)

In [ ]:
data = X_test
targets = y_test

test_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=250)

In [ ]:
X_train = train_gen[0][0]
y_train = train_gen[0][1]
X_test = test_gen[0][0]
y_test = test_gen[0][1]

In [ ]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (510, 240, 31)
y train shape: (510, 31, 2)
x test shape: (250, 240, 31)
y test shape: (250, 31, 2)


In [ ]:
X_train = X_train.transpose((0,2,1))
X_train = np.reshape(X_train, (510 * 31, 240))
y_train = np.reshape(y_train, (510 * 31, 2))

In [ ]:
# Reshaping X_train for efficient modelling
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (15810, 240, 1)
y train shape: (15810, 2)
x test shape: (250, 240, 31)
y test shape: (250, 31, 2)


In [ ]:
# expected input data shape: (batch_size, timesteps, data_dim)

# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
# regressor.add(LSTM(units=25, return_sequences=True, input_shape=(240, 31)))
regressor.add(LSTM(units=25, input_shape=(240, 1), dropout=0.1))

# The output layer
# regressor.add(LSTM(25))
regressor.add(Dense(2, activation='softmax'))
# regressor.add(Reshape((31,2)))
# regressor.add(softmax(axis = -1))
# regressor.add(Lambda(lambda  x: softmax(x, axis = -1)))
# Compiling the RNN
regressor.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
# regressor.compile(loss='mean_squared_error',
#               optimizer='rmsprop',
#               metrics=['accuracy'])
# Fitting to the training set
regressor.fit(X_train,y_train,epochs=1000,batch_size=128, validation_split=0.2, callbacks = [EarlyStopping(monitor='val_loss', patience=10),
             ModelCheckpoint(filepath='../model/LSTM/best_model.h5', monitor='val_acc', save_best_only=True)])

Train on 12648 samples, validate on 3162 samples
Epoch 1/1000
12648/12648 [==============================] - 20s 2ms/step - loss: 0.6929 - acc: 0.5132 - val_loss: 0.6930 - val_acc: 0.5057
Epoch 2/1000
12648/12648 [==============================] - 19s 1ms/step - loss: 0.6927 - acc: 0.5157 - val_loss: 0.6925 - val_acc: 0.5215
Epoch 3/1000
12648/12648 [==============================] - 19s 1ms/step - loss: 0.6927 - acc: 0.5155 - val_loss: 0.6924 - val_acc: 0.5183
Epoch 4/1000
12648/12648 [==============================] - 19s 1ms/step - loss: 0.6925 - acc: 0.5119 - val_loss: 0.6927 - val_acc: 0.5168
Epoch 5/1000
12648/12648 [==============================] - 19s 1ms/step - loss: 0.6925 - acc: 0.5137 - val_loss: 0.6925 - val_acc: 0.5225
Epoch 6/1000
12648/12648 [==============================] - 20s 2ms/step - loss: 0.6926 - acc: 0.5157 - val_loss: 0.6926 - val_acc: 0.5190
Epoch 7/1000
12648/12648 [==============================] - 20s 2ms/step - loss: 0.6924 - acc: 0.5162 - val_loss: 0.6

In [ ]:
# Evaluating our model
predicted = regressor.predict(X_test)

In [ ]:
label = predicted > 0.5
label = label * 1 # Convert boolean to int

In [ ]:
(sum(y_test[:, :, 1] == label[:, :, 1])/(y_test.size/2)).sum()

In [ ]:
model_name = '../model/LSTM/my_model3.h5'
regressor.save(model_name)  # creates a HDF5 file 'my_model.h5'
del regressor  # deletes the existing model

In [ ]:
from keras.models import load_model
from keras.activations import softmax

# returns a compiled model
# identical to the previous one
regressor1 = load_model(model_name, custom_objects={"softmax": softmax})

In [ ]:
predicted_load = regressor1.predict(X_test)
label_load = predicted_load > 0.5
label_load = label_load * 1 # Convert boolean to int
(sum(y_test[:, :, 1] == label_load[:, :, 1])/(y_test.size/2)).sum()